In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms, datasets
from collections import Counter
import joblib
from tqdm.notebook import tqdm
import time
import os

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
study = joblib.load("your own path of hyperparameters")
best_params = study.best_params

In [ ]:
lr = best_params['lr']
batch_size = best_params['batch_size']
optimizer_choice = best_params['optimizer']
weight_decay = best_params['weight_decay']
dropout_rate = best_params['dropout']
epochs = best_params["epochs"]

In [ ]:
train_dir = "your own path of training dataset"  

transform = transforms.Compose([transforms.Resize((456, 456)),
                                      transforms.RandomHorizontalFlip(), 
                                      transforms.RandomVerticalFlip(),
                                      transforms.ToTensor(), 
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                     ])

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
num_classes = 2  

model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
for param in model.parameters():
    param.requires_grad = False

in_features = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Dropout(p=best_params['dropout']),
    nn.Linear(in_features, num_classes)
)
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
if best_params['optimizer'] == 'Adam':
    optimizer = optim.Adam(model.classifier.parameters(), lr=best_params['lr'], weight_decay=best_params['weight_decay'])
else:
    optimizer = optim.SGD(model.classifier.parameters(), lr=best_params['lr'], weight_decay=best_params['weight_decay'], momentum=0.9)

In [ ]:
import copy

num_epochs = best_params['epochs']
early_stopping_patience = 5

best_loss = float('inf')
epochs_no_improve = 0
best_model_wts = copy.deepcopy(model.state_dict())  

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True, ncols=100):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        if torch.isnan(loss):
            print("Loss is NaN. Skipping batch.")
            continue

        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f"[{epoch+1}/{num_epochs}] Loss: {epoch_loss:.4f}")

    if epoch_loss < best_loss:
        best_loss = epoch_loss
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save(model.state_dict(), "best_model.pth")
        print(f"Loss improved. Model saved.")
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        print(f"No improvement. Patience: {epochs_no_improve}/{early_stopping_patience}")
        if epochs_no_improve >= early_stopping_patience:
            print("Early stopping triggered.")
            break

model.load_state_dict(best_model_wts)